In [16]:
import json
import requests
import pandas as pd
from funcs import utils, paths
from funcs.data_processing import stage1_processing

In [6]:
output_path = utils.find_data_root() / "output" / "pubtator"
assert output_path.exists()

batch_request_dir = output_path / "batch_requests"
assert batch_request_dir.exists()

batch_info_path = output_path / "batch_request_info.json"
assert batch_info_path.exists()

In [9]:
with batch_info_path.open("r") as f:
    batch_info = json.load(f)
print(len(batch_info))

133


In [10]:
IDX = 0

batch = batch_info[IDX]
print(batch)

{'batch': 'ebi', 'idx': 0, 'start_idx': 0, 'end_idx': 200, 'terms': ['gonarthrosis', 'psoriatic and enteropathic arthropathies', 'pain associated with micturition', 'other mood', 'preterm delivery', 'injury of nerves at forearm level', 'injury of nerves at wrist and hand level', 'gastritis and duodenitis', 'problems related to physical environment', 'pre existing hypertension complicating pregnancy childbirth and the puerperium', 'pre existing hypertensive disorder with superimposed proteinuria', 'injury of blood vessels at wrist and hand level', 'superficial injuries involving multiple body regions', 'open wounds involving multiple body regions', 'other injuries involving multiple body regions', 'effects of air pressure and water pressure', 'sequelae of injuries of neck and trunk', 'sequelae of injuries involving multiple and unspecified body regions', 'other injuries of spine and trunk level unspecified', 'personal history of malignant neoplasm', 'seropositive rheumatoid arthritis', 

In [30]:
batch_idx = "{batch}-{idx}".format(batch=batch["batch"], idx=batch["idx"])
with (batch_request_dir / f"{batch_idx}.txt").open() as f:
    batch_raw_data = f.read()
print(batch_raw_data)
batch_raw_split = batch_raw_data.split("\n")

00000|t|gonarthrosis -*- psoriatic and enteropathic arthropathies -*- pain associated with micturition -*- other mood -*- preterm delivery -*- injury of nerves at forearm level -*- injury of nerves at wrist and hand level -*- gastritis and duodenitis -*- problems related to physical environment -*- pre existing hypertension complicating pregnancy childbirth and the puerperium -*- pre existing hypertensive disorder with superimposed proteinuria -*- injury of blood vessels at wrist and hand level -*- superficial injuries involving multiple body regions -*- open wounds involving multiple body regions -*- other injuries involving multiple body regions -*- effects of air pressure and water pressure -*- sequelae of injuries of neck and trunk -*- sequelae of injuries involving multiple and unspecified body regions -*- other injuries of spine and trunk level unspecified -*- personal history of malignant neoplasm -*- seropositive rheumatoid arthritis -*- hypertrophic disorders of skin -*- other

In [31]:
text_prefix = "00000|t|"
raw_text = batch_raw_split[0][len(text_prefix):]
print(raw_text)

gonarthrosis -*- psoriatic and enteropathic arthropathies -*- pain associated with micturition -*- other mood -*- preterm delivery -*- injury of nerves at forearm level -*- injury of nerves at wrist and hand level -*- gastritis and duodenitis -*- problems related to physical environment -*- pre existing hypertension complicating pregnancy childbirth and the puerperium -*- pre existing hypertensive disorder with superimposed proteinuria -*- injury of blood vessels at wrist and hand level -*- superficial injuries involving multiple body regions -*- open wounds involving multiple body regions -*- other injuries involving multiple body regions -*- effects of air pressure and water pressure -*- sequelae of injuries of neck and trunk -*- sequelae of injuries involving multiple and unspecified body regions -*- other injuries of spine and trunk level unspecified -*- personal history of malignant neoplasm -*- seropositive rheumatoid arthritis -*- hypertrophic disorders of skin -*- other infesta

In [29]:
from io import StringIO

ner_res = pd.read_csv(
    StringIO(
        "\n".join(
            batch_raw_split[2:]
        )
    ),
    names=["start_pos", "end_pos", "text", "ent_type", "ent"],
    index_col=0,
    sep="\t"
)
ner_res

,start_pos,end_pos,text,ent_type,ent
0,0,66,gonarthrosis -*- psoriatic and enteropathic ar...,Disease,MESH:D015535
0,218,227,gastritis,Disease,MESH:D005756
0,232,242,duodenitis,Disease,MESH:D004382
0,305,317,hypertension,Disease,MESH:D006973
0,360,370,puerperium,Chemical,NaN
...,...,...,...,...,...
0,7453,7470,diabetes mellitus,Disease,MESH:D003920
0,7491,7528,diabetes mellitus -*- viral infection,Disease,MESH:D003920
0,7532,7543,unspecified,Species,32644
0,7559,7635,cataract -*- cytomegaloviral disease -*- insul...,Disease,MESH:D003922


In [38]:
with pd.option_context("display.min_rows", 100):
    print(ner_res)

    start_pos  end_pos                                               text  \
0           0       66  gonarthrosis -*- psoriatic and enteropathic ar...   
0         218      227                                          gastritis   
0         232      242                                         duodenitis   
0         305      317                                       hypertension   
0         360      370                                         puerperium   
0         388      409                              hypertensive disorder   
0         428      450                             proteinuria -*- injury   
0         786      797                                        unspecified   
0         855      866                                        unspecified   
0         927      974    rheumatoid arthritis -*- hypertrophic disorders   
0        1010     1018                                           fibrosis   
0        1023     1059               cirrhosis of liver -*- polyarthrosis   

In [35]:
raw_text[7559:7635]

'cataract -*- cytomegaloviral disease -*- insulin dependent diabetes mellitus'